In [9]:
# mount drive
from google.colab import drive
drive.mount("/drive")

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [10]:
# imports for plots
from plotly import graph_objects as go
import plotly.express as px
import plotly.io as pio

pio.templates.default = "plotly_white"

In [11]:
import requests
import pandas as pd


class LocationWeather:
  def __init__(self, lat, lon, start_date="19810101", end_date="20210131"):

    # to call API
    URL = "https://power.larc.nasa.gov/cgi-bin/v1/DataAccess.py?"+\
    "&request=execute&identifier=SinglePoint&"+\
    "parameters=PRECTOT,WS2M,RH2M,T2MDEW,T2M_MAX,T2M_MIN,ALLSKY_SFC_SW_DWN,T2M&"+\
    f"startDate={start_date}&endDate={end_date}&userCommunity=AG&"+\
    f"tempAverage=DAILY&outputList=CSV&lat={lat}&lon={lon}&user=DAV"

    # network request
    self.res = requests.get(URL).json()

    # build weather data frame
    # dict
    self.weather_dict = {}
    res_reduced = self.res["features"][0]["properties"]['parameter']

    # date col
    # all dic obj are { "date": val }
    translate_to_date = LocationWeather.translate_to_date
    self.weather_dict["DATE"] = [translate_to_date(d) for d in res_reduced["PRECTOT"].keys()]

    # build weather col
    for key in res_reduced:
      self.weather_dict[key] = list(res_reduced[key].values())

    # final data frame object
    self.weather_data_frame = pd.DataFrame(self.weather_dict)
    return

  @staticmethod
  # transform date year_month_day --> date object
  # date of 20200114 --> 2020-01-14
  def translate_to_date(s_date):

    # split string
    year = int(s_date[0:4])
    month = int(s_date[4:6])
    day = int(s_date[6:])

    return pd.Timestamp(year=year, month=month, day=day)

In [12]:
# # test
# loc_test = LocationWeather(lat=10, lon=12)

# loc_test.weather_data_frame

In [13]:
# arr of location center around Sidi SLIMAN

import numpy as np

# sidi SLIMAN
center_point = np.array([34.269355, -5.938411], dtype=float)
perimeter_point = np.array([34.26860, -5.99298], dtype=float)

# center and perimeter points were choosen such that radius = 5km
R = np.sqrt((center_point - perimeter_point) @ (center_point - perimeter_point))

# to generate locations around sidi SLIMAN
# nb point along an r-axis
# nb_points is nb_points * 16 - 16 + 1 
def generate_locations(nb_points):
  # axis
  arr_r = np.linspace(0, R, nb_points)[1:]
  arr_theta = np.linspace(0, 2 * np.pi, 16)

  # func to gen single loc
  func = lambda r, theta: center_point + r * np.array([np.cos(theta), np.sin(theta)])

  polar_grid = []
  for r in arr_r:
    for theta in arr_theta:
      loc = func(r, theta)

      polar_grid.append(loc)
  
  # add center
  polar_grid.append(center_point)

  return polar_grid

In [18]:
# shape of the polar grid
x, y = np.array(generate_locations(3)).T

fig = go.Figure(
    data=[
            go.Scatter(x=x, y=y, mode="markers")
])


fig.update_layout(
    height=500,
    width=500
)
fig.show()

In [15]:
# load data

class WholeLocationWeather:
  def __init__(self, arr_locations):
    self.dict_locations_weather = {}

    # get and store weather of locations
    for loc in arr_locations:
      lat, lon = loc
      key = f"{lat};{lon}"

      self.dict_locations_weather[key] = LocationWeather(lat, lon).weather_data_frame
    return

  # change path of directory
  # to change where csv are saved
  def save_as_csv(self):
    for loc in self.dict_locations_weather.keys():
      # path name
      path = "/drive/My Drive/Colab Notebooks/AgriEdge/datasets/" + f"{loc}.csv"
      #path = f"data_dir/{loc}.csv"

      self.dict_locations_weather[loc].to_csv(path, index=False)
    return

In [16]:
# __main__
# generate arr of locations
arr_locs = generate_locations(nb_points=0)

# 
whole_data = WholeLocationWeather(arr_locs)

In [17]:
whole_data.save_as_csv()